In [4]:
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 29.0 MB/s eta 0:00:00


In [5]:
from pymongo import MongoClient
from tqdm import tqdm
import json
import pandas as pd
import numpy as np

In [2]:
import json
with open('my_dict.json', 'r') as file:
    data = json.load(file)

In [6]:
client = MongoClient('mongodb://49.13.173.177:27020/')
embs = client.sponsoredbye.embeddings

embeddings = []
labels = []

for n in tqdm(data['4']):
  temp_embs = []
  cursor = embs.find({'videoID':n})
  if cursor[0]['label'] == False:
    continue
  l = cursor[0]['label']
  e =  cursor[0]['embeddings']
  for m in e:
    temp_embs.append(m['embedding'])
  embeddings.append(temp_embs)
  labels.append(l)


100%|██████████| 1981/1981 [12:08<00:00,  2.72it/s]


#Logistic Classifier

### Extract embeddings and labels + padding

In [16]:
X_train = [elm for x in embeddings for elm in x]
y_train = [elm for x in labels for elm in x]

In [9]:
from sklearn.linear_model import LogisticRegression

In [18]:
logistic_class = LogisticRegression(random_state=0
                                     ,n_jobs=-1
                                     ,max_iter = 10000
                                     ,solver='lbfgs'
                                     ,multi_class='auto'
                                     ,C = 1 )
logistic_class.fit(X_train , y_train)

LogisticRegression(C=1, max_iter=10000, n_jobs=-1, random_state=0)

In [19]:
x_test = []
y_test = []
for n in tqdm(data['15']):
  temp_embs = []
  cursor = embs.find({'videoID':n})
  if cursor[0]['label'] == False:
    continue
  l = cursor[0]['label']
  e =  cursor[0]['embeddings']
  for m in e:
    temp_embs.append(m['embedding'])
  x_test.append(temp_embs)
  y_test.append(l)

x_test = [elm for x in x_test for elm in x]
y_test = [elm for x in y_test for elm in x]

100%|██████████| 1978/1978 [12:51<00:00,  2.57it/s]


In [20]:
predictions = logistic_class.predict(x_test)

In [31]:
from sklearn.metrics import f1_score
f1_score(y_test, predictions, average= 'micro')

0.6890818835834429

In [30]:
results = {'real': y_test,
           'prediction': predictions.tolist()
}

with open('logictic_results.json', 'w') as file:
    json.dump(results, file, indent=4)
